In [30]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.sampler import RandomSampler
from warmup_scheduler import GradualWarmupScheduler
import albumentations
from work.utils.dataset import RGB2LABTransform, PandasDataset
from work.utils.models import EfficientNet
from work.utils.train import train_model
from work.utils.metrics import model_checkpoint
import random

In [31]:
backbone_model = 'efficientnet-b0'
pretrained_model = {
    backbone_model: '../pre-trained-models/efficientnet-b0-08094119.pth'
}

data_dir = '../../dataset'
images_dir = os.path.join(data_dir, 'tiles')

df_train = pd.read_csv(f"../data/train_5fold.csv")

In [32]:
seed = 42
shuffle = True
batch_size = 2
num_workers = 4
output_classes = 5
init_lr = 3e-4
warmup_factor = 10
warmup_epochs = 1
n_epochs = 15
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Using device: cuda


In [33]:
loss_function = nn.BCEWithLogitsLoss()
transforms = albumentations.Compose([
    RGB2LABTransform(p=1.0),
    albumentations.Transpose(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    albumentations.HorizontalFlip(p=0.5),
    # albumentations.Normalize(mean=(50, 0, 0), std=(50, 128, 128))
])

val_transform = albumentations.Compose([
    RGB2LABTransform(p=1.0)
])

In [34]:
df_train.columns = df_train.columns.str.strip()

train_indexes = np.where((df_train['fold'] != 3))[0]
valid_indexes = np.where((df_train['fold'] == 3))[0]

train = df_train.loc[train_indexes]
valid = df_train.loc[valid_indexes]

train_dataset = PandasDataset(images_dir, train, transforms=transforms)
valid_dataset = PandasDataset(images_dir, valid, transforms=val_transform)

In [35]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=2, num_workers=num_workers, sampler = RandomSampler(train_dataset)
)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=2, num_workers=num_workers, sampler = RandomSampler(valid_dataset)
)

In [36]:
model = EfficientNet(
    backbone=backbone_model,
    output_dimensions=output_classes,
    pre_trained_model=pretrained_model
)
model = model.to(device)

Loaded pretrained weights for efficientnet-b0


In [37]:
optimizer = optim.Adam(model.parameters(), lr = init_lr / warmup_factor)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs - warmup_epochs)
scheduler = GradualWarmupScheduler(optimizer, multiplier = warmup_factor, total_epoch = warmup_epochs, after_scheduler=scheduler_cosine)
save_path = f'models/with-noise-lab.pth'

print(optimizer, scheduler_cosine)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 2.9999999999999997e-05
    lr: 2.9999999999999997e-05
    maximize: False
    weight_decay: 0
) <torch.optim.lr_scheduler.CosineAnnealingLR object at 0x746f0152d510>


In [38]:
train_model(
    model=model,
    epochs=n_epochs,
    optimizer=optimizer,
    scheduler=scheduler,
    train_dataloader=train_loader,
    valid_dataloader=valid_loader,
    checkpoint=model_checkpoint,
    device=device,
    loss_function=loss_function,
    path_to_save_metrics="logs/with-noise-lab.txt",
    path_to_save_model=save_path
)

Epoch 1/15



100%|██████████| 903/903 [04:02<00:00,  3.72it/s]


Salvando o melhor modelo... 0.0 -> 0.6903065525497069
Epoch 2/15



100%|██████████| 903/903 [03:41<00:00,  4.08it/s]


Epoch 3/15



...

Epoch 14/15



100%|██████████| 903/903 [03:31<00:00,  4.28it/s]


Epoch 15/15



100%|██████████| 903/903 [03:31<00:00,  4.28it/s]

Salvando o melhor modelo... 0.8483710943746832 -> 0.8499759192540549
